In [1]:
import pandas as pd
import urllib
import json
import requests
import os
host = 'https://www.encodeproject.org'
experiments = "https://www.encodeproject.org/experiments/"
sample_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_file_manifest_from_website.txt'
control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_control_file_manifest_from_website.txt'
combined_sample_and_control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_manifest.txt'
out_dir = '/projects/ps-yeolab3/encode/rnaseq/differential_expression/'
out_dir2 = '/oasis/tscc/scratch/bay001/encode/data/'
to_download = '/home/bay001/projects/encode/permanent_data/to_download_9-14-2016.sh'
to_download2 = '/home/bay001/projects/encode/permanent_data/to_download2_9-14-2016.sh'

### You'll need the metadata file downloaded from the encodeproject.org website

In [2]:
def star_or_tophat(link):
    if os.path.basename(link).find('star') > -1:
        return 'STAR'
    if os.path.basename(link).find('tophat') > -1:
        return 'TOPHAT'
    return 'ambiguous'

In [3]:
X = pd.read_table('/home/bay001/projects/encode/permanent_data/metadata_from_encodeprojectorg_9-11-2016.tsv')
X.head(2)
print(X.shape)

(6678, 48)


In [4]:
X = X[X['Output type']=='alignments']
X = X[X['Lab']=="ENCODE Processing Pipeline"]
print(X.shape)

(3816, 48)


In [5]:
Y = pd.read_table('/home/bay001/projects/encode/permanent_data/rna_seq_file_manifest_from_website.txt')
Y.drop_duplicates(inplace=True)
Y.shape

(3432, 11)

In [10]:
X

,File accession,File format,Output type,Experiment accession,Assay,Biosample term id,Biosample term name,Biosample type,Biosample life stage,Biosample sex,...,md5sum,File download URL,Assembly,Platform,Controlled by,File Status,Audit WARNING,Audit INTERNAL_ACTION,Audit NOT_COMPLIANT,Audit ERROR
2,ENCFF946IIV,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,d1376c1eb68678c56a0dc8bdcfcb6257,https://www.encodeproject.org/files/ENCFF946II...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
4,ENCFF796TJR,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,ec2f056b85b9204e1e6a34f13814d334,https://www.encodeproject.org/files/ENCFF796TJ...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
6,ENCFF391JUX,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,fbf8ab86616501bf2a234c3721d7d672,https://www.encodeproject.org/files/ENCFF391JU...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
7,ENCFF478PRI,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,1dc467e36454532e9682f64bec6ef80a,https://www.encodeproject.org/files/ENCFF478PR...,hg19,NaN,NaN,released,NaN,NTR assay,NaN,NaN
8,ENCFF516JSK,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,a3c9fc10ca954d4c4074f0e4f82123e5,https://www.encodeproject.org/files/ENCFF516JS...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
9,ENCFF964IGW,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,7c6a5a8c7dfffc8e76f675e765e9fd05,https://www.encodeproject.org/files/ENCFF964IG...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
10,ENCFF754BYO,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,dab0bede1f617add3cf0b7fa4c378f2b,https://www.encodeproject.org/files/ENCFF754BY...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
13,ENCFF333QYQ,bam,alignments,ENCSR386YEV,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,9b250763713af17aa230b166c8a6b8ee,https://www.encodeproject.org/files/ENCFF333QY...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
16,ENCFF677ZIS,bam,alignments,ENCSR844QNT,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,c26f14d0bfe0d0325347907f1f3bf0bd,https://www.encodeproject.org/files/ENCFF677ZI...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN
17,ENCFF472UYI,bam,alignments,ENCSR844QNT,shRNA knockdown followed by RNA-seq,EFO:0002067,K562,immortalized cell line,adult,female,...,65cf2912a2b928a8614b39b12d05f3c9,https://www.encodeproject.org/files/ENCFF472UY...,GRCh38,NaN,NaN,released,NaN,NTR assay,NaN,NaN


### This block scrolls through the Encode webpage JSON code to extract the correct locations / links to the KD bam files. Links to control bam files are generated further below. For now, names of control bam files are stored in a dictionary, with the accession number as key and the bam file name as value

### This should follow the schema from (as of 9-8-2016): https://github.com/ENCODE-DCC/encoded/blob/master/src/encoded/schemas/file.json

In [25]:
samples = list()
links = list()
controls = list()
error = list()
havenocontrol = list()
i = 0
for h in X['Experiment accession'].dropna().drop_duplicates(): # queries every experiment
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for f in data['files']: # for every file associated with the accession
            
            i = i + 1
            
            if ((host+f['href']).find('ENCFF840VCF') != -1):
                print(f['replicate']['experiment']['target']['name'])
            if ((host+f['href']).endswith('bam') & 
            (f['output_type'] == u'alignments') &
            (f['lab'][u'name'] == 'encode-processing-pipeline')):
                try:
                    if(i%100==0):
                        print('.'),
                    samples.append({'expt_name':f['replicate']['experiment']['target']['name'],
                                    'expt_cell_line':f['replicate'][u'library']['biosample']['biosample_term_name'],
                                    'biological_replicate_number':f['replicate']['biological_replicate_number'],
                                    'submitted_expt_file_name':f['submitted_file_name'],
                                    'expt_file_name':os.path.basename(f['href']),
                                    'assembly':f['assembly'],
                                    'expt_url':host+f['href'],
                                    'RNASEQ_ControlENC':data['possible_controls'][0]['accession'],
                                    'RNASEQ_ENCODEAccID':h,
                                    'aligner': star_or_tophat(f['submitted_file_name']),
                                    'expt_md5sum': f['md5sum']
                                   })
                    controls.append(data['possible_controls'][0]['accession'])
                except IndexError:
                    havenocontrol.append(h)
                links.append(host+f['href'])
        
samples_df = pd.DataFrame(samples)
samples_df.to_csv(sample_manifest,
                  sep='\t',
                  index=None)
samples_df

. . . . . . . . . . . . . . . . . . . . . Non-specific target control-human
. . . . . . . . . .

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name
0,ENCSR667PLJ,ENCSR386YEV,STAR,hg19,1,K562,ENCFF946IIV.bam,d1376c1eb68678c56a0dc8bdcfcb6257,CCAR1-human,https://www.encodeproject.org/files/ENCFF946II...,/long-RNA-seq/runs/hg19/ENCSR386YEV/rep1_1/ENC...
1,ENCSR667PLJ,ENCSR386YEV,STAR,hg19,2,K562,ENCFF796TJR.bam,ec2f056b85b9204e1e6a34f13814d334,CCAR1-human,https://www.encodeproject.org/files/ENCFF796TJ...,/long-RNA-seq/runs/hg19/ENCSR386YEV/rep2_1/ENC...
2,ENCSR667PLJ,ENCSR386YEV,TOPHAT,hg19,1,K562,ENCFF391JUX.bam,fbf8ab86616501bf2a234c3721d7d672,CCAR1-human,https://www.encodeproject.org/files/ENCFF391JU...,/long-RNA-seq/posted/hg19/2015-08-26/ENCSR386Y...
3,ENCSR667PLJ,ENCSR386YEV,TOPHAT,hg19,2,K562,ENCFF478PRI.bam,1dc467e36454532e9682f64bec6ef80a,CCAR1-human,https://www.encodeproject.org/files/ENCFF478PR...,/long-RNA-seq/posted/hg19/2015-08-26/ENCSR386Y...
4,ENCSR667PLJ,ENCSR386YEV,TOPHAT,GRCh38,1,K562,ENCFF516JSK.bam,a3c9fc10ca954d4c4074f0e4f82123e5,CCAR1-human,https://www.encodeproject.org/files/ENCFF516JS...,/long-RNA-seq/runs/GRCh38/ENCSR386YEV/rep1_1/E...
5,ENCSR667PLJ,ENCSR386YEV,TOPHAT,GRCh38,2,K562,ENCFF964IGW.bam,7c6a5a8c7dfffc8e76f675e765e9fd05,CCAR1-human,https://www.encodeproject.org/files/ENCFF964IG...,/long-RNA-seq/runs/GRCh38/ENCSR386YEV/rep2_1/E...
6,ENCSR667PLJ,ENCSR386YEV,STAR,GRCh38,1,K562,ENCFF754BYO.bam,dab0bede1f617add3cf0b7fa4c378f2b,CCAR1-human,https://www.encodeproject.org/files/ENCFF754BY...,/long-RNA-seq/runs/GRCh38/ENCSR386YEV/rep1_1/E...
7,ENCSR667PLJ,ENCSR386YEV,STAR,GRCh38,2,K562,ENCFF333QYQ.bam,9b250763713af17aa230b166c8a6b8ee,CCAR1-human,https://www.encodeproject.org/files/ENCFF333QY...,/long-RNA-seq/runs/GRCh38/ENCSR386YEV/rep2_1/E...
8,ENCSR164MUK,ENCSR844QNT,TOPHAT,GRCh38,2,K562,ENCFF677ZIS.bam,c26f14d0bfe0d0325347907f1f3bf0bd,PPP1R8-human,https://www.encodeproject.org/files/ENCFF677ZI...,/long-RNA-seq/runs/GRCh38/ENCSR844QNT/rep2_1/E...
9,ENCSR164MUK,ENCSR844QNT,TOPHAT,GRCh38,1,K562,ENCFF472UYI.bam,65cf2912a2b928a8614b39b12d05f3c9,PPP1R8-human,https://www.encodeproject.org/files/ENCFF472UY...,/long-RNA-seq/runs/GRCh38/ENCSR844QNT/rep1_1/E...


In [26]:
# Every experiment: contains 3432 bam files
samples_df.shape

(3432, 11)

In [27]:
# download the gene quantifications

# remove duplicates from this list
havenocontrol = list(set(havenocontrol))
controls = list(set(controls))
len(controls)

46

In [28]:
print(set(controls) - set(havenocontrol))
print(set(havenocontrol) - set(controls)) # these rbps don't have controls online.


set([])
set(['ENCSR703QFF', 'ENCSR096BEN'])


In [29]:
samples = list()
error = list()
for h in controls:
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for i in range(0,len(data['files'])):
            # for key, value in data['files'][i].iteritems():
            #     print(key)
            # print(data['files'][i].iteritems())
            # print( data['files'][i]['replicate']['biological_replicate_number'])
            if ((host+data['files'][i]['href']).endswith('bam') & 
            (data['files'][i]['output_type'] == u'alignments') &
            (data['files'][i]['lab'][u'name'] == 'encode-processing-pipeline')):
                """if(list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0] != data['possible_controls'][0]['accession']):
                    error.append("WARNING: CTRL VS AFF CHANGED: {}:{}:{}".format(
                        h,
                        list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0],
                        data['possible_controls'][0]['accession']))"""
                samples.append({'ctrl_name':data['files'][i]['replicate']['experiment']['target']['name'],
                                'ctrl_cell_line':data['files'][i]['replicate'][u'library']['biosample']['biosample_term_name'],
                                'biological_replicate_number':data['files'][i]['replicate']['biological_replicate_number'],
                                'submitted_ctrl_file_name':data['files'][i]['submitted_file_name'],
                                'ctrl_file_name':os.path.basename(data['files'][i]['href']),
                                'assembly':data['files'][i]['assembly'],
                                'ctrl_url':host+data['files'][i]['href'],
                                'aligner': star_or_tophat(data['files'][i]['submitted_file_name']),
                                'RNASEQ_ControlENC':h,
                                'ctrl_md5sum': data['files'][i]['md5sum']})
                links.append(host+data['files'][i]['href'])
                # controls[data['accession']] = data['possible_controls'][0]['accession']
samples_df = pd.DataFrame(samples)
samples_df.to_csv(control_manifest,
                  sep='\t',
                  index=None)
samples_df

,RNASEQ_ControlENC,aligner,assembly,biological_replicate_number,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR572FFX,STAR,hg19,1,K562,ENCFF310XPL.bam,0dc58433dc4cdf54e756c39a970ea285,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF310XP...,/long-RNA-seq/runs/hg19/ENCSR572FFX/rep1_1/ENC...
1,ENCSR572FFX,STAR,hg19,2,K562,ENCFF301VBU.bam,2305a2e8c127e9e951d56fc0287c6ca6,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF301VB...,/long-RNA-seq/runs/hg19/ENCSR572FFX/rep2_1/ENC...
2,ENCSR572FFX,TOPHAT,hg19,1,K562,ENCFF671NCT.bam,175f9832bdb8a5fb71761aa23064dc44,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF671NC...,/long-RNA-seq/posted/hg19/2015-08-26/ENCSR572F...
3,ENCSR572FFX,TOPHAT,hg19,2,K562,ENCFF035JPJ.bam,893afa6ca22411c627edb7131de679a2,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF035JP...,/long-RNA-seq/posted/hg19/2015-08-26/ENCSR572F...
4,ENCSR572FFX,TOPHAT,GRCh38,1,K562,ENCFF148ERF.bam,d6158f30f22d75326eed6b9297149969,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF148ER...,/long-RNA-seq/runs/GRCh38/ENCSR572FFX/rep1_1/E...
5,ENCSR572FFX,STAR,GRCh38,1,K562,ENCFF568DGW.bam,c40e5d4a835348467255743693e680b0,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF568DG...,/long-RNA-seq/runs/GRCh38/ENCSR572FFX/rep1_1/E...
6,ENCSR572FFX,TOPHAT,GRCh38,2,K562,ENCFF802CAV.bam,d8c36791c0299f4370b90828dc079872,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF802CA...,/long-RNA-seq/runs/GRCh38/ENCSR572FFX/rep2_1/E...
7,ENCSR572FFX,STAR,GRCh38,2,K562,ENCFF691JCT.bam,f08e1948ce39333c5267d751e3d7a893,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF691JC...,/long-RNA-seq/runs/GRCh38/ENCSR572FFX/rep2_1/E...
8,ENCSR344XID,TOPHAT,hg19,1,K562,ENCFF232CRW.bam,2652bc908cc8d4f326be65f1acce1f21,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF232CR...,/long-RNA-seq/runs/hg19/ENCSR344XID/rep1_1/ENC...
9,ENCSR344XID,TOPHAT,hg19,2,K562,ENCFF615FRB.bam,ed825a0541ecccc101fbaad960856474,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF615FR...,/long-RNA-seq/runs/hg19/ENCSR344XID/rep2_1/ENC...


In [30]:
samples_df.shape

(368, 10)

In [31]:
samples_df = pd.read_table(sample_manifest)
controls_df = pd.read_table(control_manifest)

In [32]:
samples_df.head(2)

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name
0,ENCSR667PLJ,ENCSR386YEV,STAR,hg19,1,K562,ENCFF946IIV.bam,d1376c1eb68678c56a0dc8bdcfcb6257,CCAR1-human,https://www.encodeproject.org/files/ENCFF946II...,/long-RNA-seq/runs/hg19/ENCSR386YEV/rep1_1/ENC...
1,ENCSR667PLJ,ENCSR386YEV,STAR,hg19,2,K562,ENCFF796TJR.bam,ec2f056b85b9204e1e6a34f13814d334,CCAR1-human,https://www.encodeproject.org/files/ENCFF796TJ...,/long-RNA-seq/runs/hg19/ENCSR386YEV/rep2_1/ENC...


In [33]:
controls_df.head(2)

,RNASEQ_ControlENC,aligner,assembly,biological_replicate_number,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR572FFX,STAR,hg19,1,K562,ENCFF310XPL.bam,0dc58433dc4cdf54e756c39a970ea285,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF310XP...,/long-RNA-seq/runs/hg19/ENCSR572FFX/rep1_1/ENC...
1,ENCSR572FFX,STAR,hg19,2,K562,ENCFF301VBU.bam,2305a2e8c127e9e951d56fc0287c6ca6,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF301VB...,/long-RNA-seq/runs/hg19/ENCSR572FFX/rep2_1/ENC...


In [59]:
controls_df.shape

(368, 10)

### perform a pandas merge on the ControlENC, aligner, assembly, and biological replicate number

In [60]:
samples_df.shape

(3432, 11)

In [63]:
merged_df = pd.merge(samples_df,controls_df,on=['RNASEQ_ControlENC',
                                                            'aligner',
                                                            'assembly',
                                                            'biological_replicate_number',
                                                            ])
# merged_df.drop_duplicates(inplace=True)
merged_df

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR667PLJ,ENCSR386YEV,STAR,hg19,1,K562,ENCFF946IIV.bam,d1376c1eb68678c56a0dc8bdcfcb6257,CCAR1-human,https://www.encodeproject.org/files/ENCFF946II...,/long-RNA-seq/runs/hg19/ENCSR386YEV/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
1,ENCSR667PLJ,ENCSR258VGD,STAR,hg19,1,K562,ENCFF830FCO.bam,5a9e5383da78c41691dc0ef1e3a109ac,EIF3A-human,https://www.encodeproject.org/files/ENCFF830FC...,/long-RNA-seq/runs/hg19/ENCSR258VGD/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
2,ENCSR667PLJ,ENCSR208GPE,STAR,hg19,1,K562,ENCFF017QVW.bam,7906c518a6c32b240ae4a494299e4726,DDX1-human,https://www.encodeproject.org/files/ENCFF017QV...,/long-RNA-seq/runs/hg19/ENCSR208GPE/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
3,ENCSR667PLJ,ENCSR306IOF,STAR,hg19,1,K562,ENCFF622IHD.bam,072864a8d786798f939fc830c8d8248d,PARN-human,https://www.encodeproject.org/files/ENCFF622IH...,/long-RNA-seq/runs/hg19/ENCSR306IOF/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
4,ENCSR667PLJ,ENCSR164TLB,STAR,hg19,1,K562,ENCFF373UAZ.bam,2a5ee7963d8dce8327d0f1a402b6c7f4,ADAR-human,https://www.encodeproject.org/files/ENCFF373UA...,/long-RNA-seq/runs/hg19/ENCSR164TLB/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
5,ENCSR667PLJ,ENCSR712CSN,STAR,hg19,1,K562,ENCFF902UDT.bam,d88eae0b8beca8ec376f5c2549b41550,EIF4G1-human,https://www.encodeproject.org/files/ENCFF902UD...,/long-RNA-seq/runs/hg19/ENCSR712CSN/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
6,ENCSR667PLJ,ENCSR079LMZ,STAR,hg19,1,K562,ENCFF104PAD.bam,014821031fda2e4fea9f1b82d0082aef,DNAJC21-human,https://www.encodeproject.org/files/ENCFF104PA...,/long-RNA-seq/runs/hg19/ENCSR079LMZ/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
7,ENCSR667PLJ,ENCSR345VVZ,STAR,hg19,1,K562,ENCFF424VAD.bam,c65e788852c3a68b0ede12cdb1a9e02b,DHX30-human,https://www.encodeproject.org/files/ENCFF424VA...,/long-RNA-seq/runs/hg19/ENCSR345VVZ/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
8,ENCSR667PLJ,ENCSR577OVP,STAR,hg19,1,K562,ENCFF646BUZ.bam,5deaf47cc9dde281587420f67b109548,DNAJC2-human,https://www.encodeproject.org/files/ENCFF646BU...,/long-RNA-seq/runs/hg19/ENCSR577OVP/rep1_1/ENC...,K562,ENCFF678FMA.bam,341ab030c44b1a9f5ff6b5803ee37437,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF678FM...,/long-RNA-seq/runs/hg19/ENCSR667PLJ/rep1_1/ENC...
9,ENCSR667PLJ,ENCSR336DFS,STAR,hg19,1,K562,ENCFF146CHN.bam,e9095906bed15c6c785dc670d9e9f955,RBFOX2-human,https://www.encodeproject.org/files/ENCFF146CH...,/long-RNA-seq/runs/hg19/ENCSR336DFS/rep1_1/ENC...,K562,ENCFF6

In [39]:
merged_df.to_csv('/home/bay001/projects/encode/permanent_data/rna_seq_manifest_9-19-2016.txt',index=None,sep="\t")

### Skip this step if you don't want to re-generate the download script.

In [ ]:
to_download_file = open(to_download,'w')
to_download_file2 = open(to_download2,'w')
for link in list(set(links)):
    to_download_file.write("wget -O {}{} {};\n".format(out_dir, os.path.basename(link), link))
    to_download_file2.write("wget -O {}{} {};\n".format(out_dir2, os.path.basename(link), link))
to_download_file.close()
to_download_file2.close()
len(list(set(links)))

In [ ]:
! grep ENCFF840VCF $to_download2

In [ ]:
len(links)

In [ ]:
! tail /home/bay001/projects/encode/permanent_data/to_download.sh


In [ ]:
# verify that everything on the manifest looks good:
df = pd.read_table(combined_sample_and_control_manifest)
_test = df[df['expt_name']=='RBFOX2-human']
_testhg19 = _test[_test['assembly']=='hg19']
_testhg19

# Let's check out the file downloaded from ENCODEPROJECT.org

In [40]:
Y = pd.read_table('/home/bay001/projects/encode/permanent_data/metadata_from_encodeprojectorg_9-11-2016.tsv')
Y = Y[(Y['Output type'] == 'alignments') &
      (Y['Lab'] == 'ENCODE Processing Pipeline')]
Y.shape

(3816, 48)

In [42]:
# 3432 samples and 368 controls
Z = pd.read_table('/home/bay001/projects/encode/permanent_data/rna_seq_manifest_9-19-2016.txt')
Z.shape

(3432, 17)

In [55]:
# set of all bams downloaded from manifest
website_metadata = (set(Y['File accession'])) 

In [56]:
# set of all bams accounted for in our local manifest
local_manifest = (set(Z['expt_file_name'].str.replace('.bam','')).union(set(Z['ctrl_file_name'].str.replace('.bam',''))))

In [64]:
# should be equal to the number of revoked experiments * 8, so 16 is good.
len(website_metadata - local_manifest)

16